# Assignment 3
Use Naive Bayes classification for the Bank Customers Loan prediction

In [ ]:
#Reference: Chapter 8 of Book: Data Mining for Business Analytics: Python
#By Shmueli et. al.
#Step 1
# import required functionality for Naive Bayes Classification

import matplotlib.pyplot as plt
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import FunctionTransformer, Pipeline
from sklearn.preprocessing import OneHotEncoder

In [ ]:
bank_df = pd.read_csv('UniversalBank.csv')
bank_df

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


In [ ]:
# Split Education into Education_1, Education_2, Education_3 columns
#bank_df = pd.get_dummies(bank_df, columns=['Education'], dtype= float)
#bank_df Not necessary for NB

In [ ]:
trainData, holdoutData = train_test_split(bank_df, test_size=0.4, random_state=26)

## Q.A: Pivot Table
Create a pivot table for the training data with Online as a column variable, CC as a row variable, and Loan as a secondary row variable. The values inside the table should convey the count.
In Python, use panda dataframe methods melt() and pivot().

In [ ]:
# Melt the training data to prepare for pivoting
melted_trainData = pd.melt(trainData, id_vars=['Online', 'CreditCard', 'Personal Loan'], value_vars=['ID'], var_name='Variable', value_name='Count')

# Create the pivot table
pivot_table = melted_trainData.pivot_table(index=['CreditCard', 'Personal Loan'], columns='Online', values='Count', aggfunc='count', fill_value=0)

# Display the pivot table
display(pivot_table)

Online                      0     1
CreditCard Personal Loan           
0          0              762  1163
           1               75   118
1          0              316   494
           1               29    43

## Q.B. Understanding the Pivot Table: Probabilities
Consider the task of classifying a customer who owns a bank credit card and is actively using online banking services. Looking at the pivot table, what is the probability that this customer will accept the loan offer? [This is the probability of loan acceptance (Loan = 1) conditional on having a bank credit card (CC = 1) and being an active user of online banking services (Online = 1)].

## Answer to Q.B.
P(CC=1|Online=1,Loan = 1) = 43/43+494 = 0.0800

##Q.C. Create two separate pivot tables for the training data.
One will have Loan (rows) as a function of Online (columns) and the other will have Loan (rows) as a function of CC.

In [ ]:
# Create the pivot table
pivot_table = melted_trainData.pivot_table(index=['Personal Loan'], columns='Online', values='Count', aggfunc='count', fill_value=0)

# Display the pivot table
display(pivot_table)

Online,0,1
Personal Loan,,
0,1078,1657
1,104,161


In [ ]:
# Create the pivot table
pivot_table = melted_trainData.pivot_table(index=['Personal Loan'], columns='CreditCard', values='Count', aggfunc='count', fill_value=0)

# Display the pivot table
display(pivot_table)

CreditCard,0,1
Personal Loan,,
0,1925,810
1,193,72


##Q.D. Compute the following quantities [P(A | B) means “the probability ofA given B”]:

i. P(CC = 1 | Loan = 1) (the proportion of credit card holders among the loan acceptors)
= 72/(72+193) = 0.2717

ii. P(Online = 1 | Loan = 1)
=161/(161+104) = 0.6075

iii. P(Loan = 1) (the proportion of loan acceptors)
=(75+118+29+43)/3000=0.0883

iv. P(CC = 1 | Loan = 0)
=810/(810+1925)=0.2961

v. P(Online = 1 | Loan = 0)
=1657/(1657+1078)=0.6058

vi. P(Loan = 0)
=(762+1163+316+494)/3000=0.9117

##Q.E. Use the quantities computed above to compute the naive Bayes probability P(Loan = 1 | CC = 1, Online = 1).

Ans. P(Loan = 1 | CC = 1, Online = 1) =
Numerator  =  P(CC = 1 | Loan = 1) * P(Online = 1 | Loan = 1) *  P(Loan = 1)
=>0.2717 * 0.6075 * 0.0883 = 0.0145

Denominator =[  P(CC = 1 | Loan = 1) * P(Online = 1 | Loan = 1) *  P(Loan = 1) ] + [ P(CC = 1 | Loan = 0) * P(Online = 1 | Loan = 0) *  P(Loan = 0) ]
=>[ 0.0145 ] + [ (0.2960 * 0.6058 * 0.9117) ] = 0.178

P(Loan = 1 | CC = 1, Online = 1) = Numerator/Denominator = 0.0145/0.178 = 0.0815

##Q.F. Compare this value with the one obtained from the pivot table in (B). Which is a more accurate estimate?

Ans. P(Yes) from Table B = 0.0800, and P(Yes) from Table E = 0.0815

Which is a more accurate estimate?

Ans. Since the probability value from Table B comes straight from the training dataset it is a more accurate estimate, while the Naive Bayes calculation is an approximation that assumes independence between features.

In [ ]:
#code of Naive Bayes Classifier applied to Personal Loan


In [ ]:
# Select the features and target for the Naive Bayes classifier
features = ['CreditCard', 'Online']
target = 'Personal Loan'

X_train = trainData[features]
y_train = trainData[target]
X_holdout = holdoutData[features]
y_holdout = holdoutData[target]

# Initialize and train the Naive Bayes model
naive_bayes_model = GaussianNB()
naive_bayes_model.fit(X_train, y_train)

# Predict on the holdout set
y_pred = naive_bayes_model.predict(X_holdout)

# Evaluate the model (optional, but good practice)
#from sklearn.metrics import accuracy_score, classification_report

#accuracy = accuracy_score(y_holdout, y_pred)
#print(f"Accuracy on holdout set: {accuracy}")
#print(classification_report(y_holdout, y_pred))

# Predict the probability for a customer with CreditCard=1 and Online=1
new_customer = pd.DataFrame([[1, 1]], columns=features)
prediction_proba = naive_bayes_model.predict_proba(new_customer)

print(f"\nProbability of accepting the loan for a customer with CC=1 and Online=1: {prediction_proba[0][1]:.4f}")


Probability of accepting the loan for a customer with CC=1 and Online=1: 0.0789


The number coming out in E is 0.0815 whereas the number coming out in G is 0.0789.
There is a 3% differnce which is minimal.